In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
def CF(S, r, q, sig, t, u):
    mu = np.log(S) + (r - q - sig**2/2)*t
    return np.exp(1j*mu*u - ((sig*u)**2)*t/2)

## Fast Fourier Transform

In [3]:
def FFT(S0, K, r, q, sig, T, alpha, eta, n):
    N = 2**n
    k = np.log(K)
    lda = 2*np.pi/(N*eta)
    beta = np.log(K) - lda*N/2
    df = np.exp(-r*T)
    
    km = np.zeros(N)
    x = np.zeros(N)
    v = np.arange(N) * eta
    
    for i in range(N):
        km[i] = beta + i*lda
        coeff = (alpha + 1j*v[i]) * (alpha + 1j*v[i] + 1)
        u = v[i] - (alpha+1)*1j
        
        if i == 0:
            w = eta/2
        else:
            w = eta
            
        x[i] = w * df * np.exp(-1J*beta*v[i]) * CF(S0, r, q, sig, T, u) / coeff
        
    y = np.fft.fft(x)
    C = np.zeros(N)
    for i in range(N):
        multiplier = np.exp(-alpha*km[i]) / np.pi
        C[i] = multiplier * np.real(y[i])
    
    return km, C

In [4]:
S0 = 1900
T = 0.25
vol = 0.36
r = 0.02
q = 0.0187
eta = 0.25
alpha_list = [0.4, 1.0, 1.4, 3.0]
n_list = [9, 11, 13, 15]
K_list = [2000, 2100, 2200]

print(' ')

fft_result = {}

for K in K_list:
    fft_result[K] = pd.DataFrame(columns=['n', 'alpha', 'time', 'price'])
    for alpha in alpha_list:
        for n in n_list:
            k = np.log(K)
            start_time = time.time()
            km, cT_km = FFT(S0, K, r, q, vol, T, alpha, eta, n)
            # interpolate to find the option price of interest
            cT_k = np.interp(k, km, cT_km)
            elapsed_time = time.time() - start_time
            fft_result[K].loc[-1] = [n, alpha, elapsed_time, cT_k]
            fft_result[K].index += 1

/Users/macpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: ComplexWarning: Casting complex values to real discards the imaginary part


In [5]:
for K in K_list:
    print('Strike is $' + str(K))
    print(fft_result[K].to_markdown(tablefmt="grid", showindex=False))
    print('')

Strike is $2000
+-----+---------+------------+---------+
|   n |   alpha |       time |   price |
+=====+=========+============+=========+
|   9 |     0.4 | 0.0248909  | 95.3281 |
+-----+---------+------------+---------+
|  11 |     0.4 | 0.0404711  | 95.3281 |
+-----+---------+------------+---------+
|  13 |     0.4 | 0.188746   | 95.3281 |
+-----+---------+------------+---------+
|  15 |     0.4 | 0.61977    | 95.3281 |
+-----+---------+------------+---------+
|   9 |     1   | 0.00884008 | 95.2467 |
+-----+---------+------------+---------+
|  11 |     1   | 0.0378661  | 95.2467 |
+-----+---------+------------+---------+
|  13 |     1   | 0.14397    | 95.2467 |
+-----+---------+------------+---------+
|  15 |     1   | 0.652274   | 95.2467 |
+-----+---------+------------+---------+
|   9 |     1.4 | 0.0106261  | 95.2467 |
+-----+---------+------------+---------+
|  11 |     1.4 | 0.0445912  | 95.2467 |
+-----+---------+------------+---------+
|  13 |     1.4 | 0.151814   | 95.2467 |


/Users/macpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 'showindex' is deprecated. Only 'index' will be used in a future version. Use 'index' to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


## Fractional Fast Fourier Transform

In [6]:
def fracFFT(S0, K, r, q, sig, T, alpha, eta, n, lda):
    N = 2**n
    gamma = eta*lda/(2*np.pi)

    #Choice of beta
    beta = np.log(K)-N*lda/2

    # initialize x, y, z, and cT_km
    km = np.zeros((N))
    x = np.zeros((N))
    y = np.zeros((2*N), dtype=np.complex)
    z = np.zeros((2*N), dtype=np.complex)
    cT_km = np.zeros((N)) 

    # discount factors
    df = np.exp(-r*T)

    # compute x
    nuJ = np.arange(N)*eta
    psi_nuJ = CF(S0, r, q, sig, T, nuJ-(alpha+1)*1j)/((alpha + 1j*nuJ) * (alpha + 1 + 1j*nuJ))  

    for j in range(N):  
        km[j] = beta+j*lda
        if j == 0:
            wJ = (eta/2)
        else:
            wJ = eta
        x[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ
        
    # set up y
    for i in range(N):
        y[i] = np.exp(-1j * np.pi * gamma * i**2)*x[i]
    y[N:] = 0

    # set up z
    for i in range(N):
        z[i] = np.exp(1j * np.pi * gamma * i**2)
    z[N:] = z[:N][::-1]

    # compute xi_hat
    xi_hat = np.fft.ifft(np.fft.fft(y) * np.fft.fft(z))

    # compute call prices
    for i in range(N):
        cT_km[i] = np.exp(-alpha * (beta+i) * lda) * (np.exp(-1j*np.pi*gamma*i**2)*xi_hat[i]).real / np.pi

    return km, cT_km

In [7]:
lda_frfft = 0.1
n_frfft = [6, 7, 8, 9]

frfft_result = {}

for K in K_list:
    frfft_result[K] = pd.DataFrame(columns=['n', 'alpha', 'time', 'price'])
    for alpha in alpha_list:
        for n in n_frfft:
            k = np.log(K)
            start_time = time.time()
            km, cT_km = fracFFT(S0, K, r, q, vol, T, alpha, eta, n, lda_frfft)
            cT_k = np.interp(k, km, cT_km)
            elapsed_time = time.time() - start_time
            frfft_result[K].loc[-1] = [n, alpha, elapsed_time, cT_k]
            frfft_result[K].index += 1


/Users/macpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: ComplexWarning: Casting complex values to real discards the imaginary part


In [8]:
for K in K_list:
    print('Strike is $' + str(K))
    print(frfft_result[K].to_markdown(tablefmt="grid", showindex=False))
    print('')

Strike is $2000
+-----+---------+-------------+-----------------+
|   n |   alpha |        time |           price |
+=====+=========+=============+=================+
|   6 |     0.4 | 0.00462389  |   588.889       |
+-----+---------+-------------+-----------------+
|   7 |     0.4 | 0.00276399  |    82.152       |
+-----+---------+-------------+-----------------+
|   8 |     0.4 | 0.00360012  |    48.4967      |
+-----+---------+-------------+-----------------+
|   9 |     0.4 | 0.00764012  |     0.658867    |
+-----+---------+-------------+-----------------+
|   6 |     1   | 0.00100136  |  2581.51        |
+-----+---------+-------------+-----------------+
|   7 |     1   | 0.00185895  |   140.357       |
+-----+---------+-------------+-----------------+
|   8 |     1   | 0.00334907  |     2.31901     |
+-----+---------+-------------+-----------------+
|   9 |     1   | 0.00793076  |     2.44835e-05 |
+-----+---------+-------------+-----------------+
|   6 |     1.4 | 0.000808954 | 12

/Users/macpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 'showindex' is deprecated. Only 'index' will be used in a future version. Use 'index' to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


## Cosine Method

In [9]:
def cos(S0, K, r, q, sig, T, n, a, b):
    N = 2**n
    df = np.exp(-r*T)
    
    # compute Ak
    nuJ = np.arange(N)*np.pi / (b-a)
    multiplier = np.exp(-1j*a*nuJ)
    fourier = CF(S0/K, r, q, sig, T, nuJ)
    A_hat = np.real(np.multiply(fourier, multiplier))
    
    # compute Vk
    x, phi = np.zeros(N), np.zeros(N)
    x = (np.cos(nuJ*(b-a))*np.exp(b)-np.cos(nuJ*(-a))+nuJ*np.sin(nuJ*(b-a))*np.exp(b)-nuJ*np.sin(nuJ*(-a)))/(1+nuJ**2)
    phi[0] = b
    phi[1:] = np.sin(nuJ[1:]*(b-a)) - np.sin(nuJ[1:]*(-a)) / nuJ[1:]
    V = 2*K*(x-phi)/(b-a)
    
    entry = np.multiply(A_hat, V)
    price = df * (entry[0]*0.5 + np.sum(entry[1:]))
    return price
    

In [10]:
interval_list = [[-1,1], [-4,4], [-8,8], [-12,12]]
cos_result = {}

for K in K_list:
    cos_result[K] = pd.DataFrame(columns=['n', 'interval', 'time', 'price'])
    for interval in interval_list:
        for n in [4, 6, 8, 10]:
            start_time = time.time()
            cT = cos(S0, K, r, q, vol, T, n, interval[0], interval[1])
            elapsed_time = time.time() - start_time
            cos_result[K].loc[-1] = [n, interval, elapsed_time, cT]
            cos_result[K].index += 1


In [11]:
for K in K_list:
    print('Strike is $' + str(K))
    print(cos_result[K].to_markdown(tablefmt="grid", showindex=False))
    print('')

Strike is $2000
+-----+------------+-------------+------------------+
|   n | interval   |        time |            price |
+=====+============+=============+==================+
|   4 | [-1, 1]    | 0.00266504  |     95.2468      |
+-----+------------+-------------+------------------+
|   6 | [-1, 1]    | 0.00022316  |     95.2467      |
+-----+------------+-------------+------------------+
|   8 | [-1, 1]    | 0.000361919 |     95.2467      |
+-----+------------+-------------+------------------+
|  10 | [-1, 1]    | 0.000546217 |     95.2467      |
+-----+------------+-------------+------------------+
|   4 | [-4, 4]    | 0.000295877 |    -12.5562      |
+-----+------------+-------------+------------------+
|   6 | [-4, 4]    | 0.00014329  |     95.2475      |
+-----+------------+-------------+------------------+
|   8 | [-4, 4]    | 0.000230789 |     95.2467      |
+-----+------------+-------------+------------------+
|  10 | [-4, 4]    | 0.00046587  |     95.2467      |
+-----+-----

/Users/macpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 'showindex' is deprecated. Only 'index' will be used in a future version. Use 'index' to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# test with different a and b
def ab_sensitivity(a, b):
    cos_result = {}

    for K in K_list:
        cos_result[K] = pd.DataFrame(columns=['n', 'interval', 'time', 'price'])
        for n in [4, 6, 8, 10]:
            start_time = time.time()
            cT = cos(S0, K, r, q, vol, T, n, a, b)
            elapsed_time = time.time() - start_time
            cos_result[K].loc[-1] = [n, [a,b] , elapsed_time, cT]
            cos_result[K].index += 1

    for K in K_list:
        print('Strike is $' + str(K))
        print(cos_result[K].to_markdown(tablefmt="grid", showindex=False))
        print('')


In [13]:
# can input differnt a and b to find out the sensitivity
ab_sensitivity(-1, 1)

Strike is $2000
+-----+------------+-------------+---------+
|   n | interval   |        time |   price |
+=====+============+=============+=========+
|   4 | [-1, 1]    | 0.000215769 | 95.2468 |
+-----+------------+-------------+---------+
|   6 | [-1, 1]    | 0.000176907 | 95.2467 |
+-----+------------+-------------+---------+
|   8 | [-1, 1]    | 0.000291109 | 95.2467 |
+-----+------------+-------------+---------+
|  10 | [-1, 1]    | 0.00103712  | 95.2467 |
+-----+------------+-------------+---------+

Strike is $2100
+-----+------------+-------------+---------+
|   n | interval   |        time |   price |
+=====+============+=============+=========+
|   4 | [-1, 1]    | 0.000137806 | 64.8348 |
+-----+------------+-------------+---------+
|   6 | [-1, 1]    | 0.000154018 | 64.8346 |
+-----+------------+-------------+---------+
|   8 | [-1, 1]    | 0.000175953 | 64.8346 |
+-----+------------+-------------+---------+
|  10 | [-1, 1]    | 0.000427008 | 64.8346 |
+-----+------------+--

/Users/macpro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: 'showindex' is deprecated. Only 'index' will be used in a future version. Use 'index' to silence this warning.
  app.launch_new_instance()


## Comparison and Analysis

First of all, as we increase n, all methods are using more and more time, which makes sense because the increased computation work. Under the same parameter setting, FrFFT performs faster than the normal FFT and cosine method running time is also incredibly fast (even better than the FrFFT methods).

In terms of calculation accuracy, for Fast Fourier Transform, the increasing n does not bring improvement of the result, the damping factor would affect the result more and it seems the FFT method remains somewhat stable when we increase our damping factor. For Fractional Fast Fourier Transform, the number of steps n heavily impacts the pricing results, making the result very volatile. And based on the result, smaller n will lead to stable price. Nevertheless, under same strike, n and alpha, the result obtained by FFT and FrFFT are not the same.

As for cosine method, the result is not very sensitive to setting of a and b. The appropriate range would be (-1, 1) to (-8, 8). It is quite important that a and b are symmetric about the origin so as to obtain a good result. Also,  increasing the interval length does not bring too much improvement. But it is worth mentioning that <b> when we increase the interval length, we need to increase the number of steps N accordingly, otherwise the approximation is wrong </b>

In fact, the cosine method is more sensitive to the number of steps N we are choosing (which in turn depends on the interval length we choose). Nevertheless, no matter what length we choose, when n is too small, the cosine series cannot approxiate the original function very well, thus leading to inaccurate result.